In [1]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Pkg
using IJulia
using Distributions

In [2]:
include("../Test Models/COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [3]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [4]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 3, 30)
end_date   = Date(2020, 6, 30)
travel_threshold_hours = 4.0
pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [13]:
N = length(states);
T = (end_date - start_date).value + 1

forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_admitted = forecast(
    states, start_date,  end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [6]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [14]:
demand = forecast_active * Float32(nurse_days_per_patient_day_covid);

In [8]:
nurses = n_nurses(states) * Float32(pct_nurses_available);

In [ ]:
## create arrays to store all the model output
# total amount
nurses_sent_tot = zeros(T)
shortage_tot = zeros(T)
average_load = zeros(T)
patients_sent_tot = zeros(T)
overall_shortage = zeros(T)
overall_overflow = zeros(T)


In [38]:
for t in 1:T-1
    N = 11
    contaminated_hospitals = [0,0,0,0,0,0,1,1,0,0,0]
    indices = findall(x -> x == 1, contaminated_hospitals)

    stage1_matrix = falses(N,N)

        for i in 1:N  
            for j in 1:N 
                # check that the edge is directed toward the contaminated hospital
                # if true set the matrix idx to 1 else leave it as 0 
                stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
            end
        end
        
    stage2_matrix = falses(N,N)
    for i in 1:N  
        for j in 1:N 
            # Manditorily make a fully connected graph among non-contaminated hospitals
            stage2_matrix[i,j] = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
        end
    end

    model1 = reusable_resource_allocation(
            nurses,
            zeros(Float32, size(demand[:, t:t+1])...),
            demand[:, t:t+1],
            stage1_matrix,
            obj_dir = :shortage,  
            send_new_only = false,
            sendrecieve_switch_time = 0,
            min_send_amt = 0,
            smoothness_penalty = 0,
            setup_cost = 0,
            sent_penalty = 0,
            verbose = true
        );
        
        sent_nurses_contam = value.(model1[:sent])
        indices = findall(x -> x == 1, contaminated_hospitals)
        received_by_contaminated = [sum(sent_nurses_contam[:, j, :]) for j in 1:length(contaminated_hospitals)]
        sent_to_contaminated = [sum(sent_nurses_contam[j, :, :]) for j in 1:length(contaminated_hospitals)]
        nurses_afterstage1 = nurses-sent_to_contaminated+received_by_contaminated
        
        model1_result = NurseAllocationResults.results_all(sent_nurses_contam, nurses, demand[:, t:t+1], states, start_date)
        stage1_shortage_tot = model1_result.total_shortage
        stage1_average_load = model1_result.average_load

        nurses_sent_tot[t] = sent_nurses_contam
        shortage_tot[t] = stage1_shortage_tot
        average_load[t] = stage1_average_load

        forecast_initial = forecast(
        states, start_date-Dates.Day(1), start_date-Dates.Day(1),
        level=:state,
        source=:ihme,
        forecast_type=:active,
        patient_type=:regular,
        bound_type=:mean,
        )[:]

    model2 = patient_nurse_allocation(
            beds,
            forecast_initial,#demand[:, t:t+1],#nurses_afterstage1, 
            forecast_discharged[:, t:t+1],
            forecast_admitted[:, t:t+1],
            nurses_afterstage1,
            stage2_matrix,
            los=Exponential(1000),
            verbose =true,
        );

        sent_nurses_uncontam = value.(model2[:sentnurses])
        sent_patients_uncontam = value.(model2[:sentpatients])
        indices = findall(x -> x == 0, contaminated_hospitals)
        receivedn_by_uncontaminated = [sum(sent_nurses_uncontam[:, j, :]) for j in 1:length(contaminated_hospitals)]
        sentn_to_uncontaminated = [sum(sent_nurses_uncontam[j, :, :]) for j in 1:length(contaminated_hospitals)]
        receivedp_by_uncontaminated = [sum(sent_patients_uncontam[:, j, :]) for j in 1:length(contaminated_hospitals)]
        sentp_to_uncontaminated = [sum(sent_patients_uncontam[j, :, :]) for j in 1:length(contaminated_hospitals)]

        # nurses = nurses_afterstage1-sentn_to_uncontaminated+receivedn_by_uncontaminated
        # demand[:,t+1] .+= receivedp_by_uncontaminated - sentp_to_uncontaminated 

        # nurses_sent_tot[t] += sent_nurses_uncontam
        # patients_sent_tot[t] = sent_patients_uncontam

        # model2_presult = PatientAllocationResults.results_all(sent_patients_uncontam, beds, forecast_initial, forecast_admitted[:, t:t+1], forecast_discharged[:, t:t+1], states, start_date, hospitalized_days);
        # model2_nresult = NurseAllocationResults.results_all(sent_nurses_contam, nurses_afterstage1, demand, states, start_date)

        # nurses_sent_tot[t] = stage1_nsent_tot + total_nsent_stage2
        # patients_sent_tot[t] = total_psent
        # stage2_shortage_tot = model2_nresult.total_shortage
        # stage2_average_load = model2_nresult.average_load + model2_presult.average_load
        # stage2_overflow_tot = model2_nresult.total_overflow

        # overall_shortage[t] = stage1_shortage_tot + stage2_shortage_tot
        # overall_overflow[t] = stage2_overflow_tot


        #model1_result = NurseAllocationResults.results_all(sent, nurses, demand, states, start_date)
        #stage1_nsent_tot = model1_result.total_sent 
        #stage1_shortage_tot = model1_result.total_shortage
        #stage1_average_load = model1_result.average_load

        start_date = start_date + Dates.Day(1)

    # sent_nurses_uncontam = value.(model2[:sentnurses])
    # sent_patients_uncontam = value.(model2[:sentpatient])

    #println("termination status: ", termination_status(model2))
    #println("solve time: ", round(solve_time(model2), digits=3), "s")
    #println("objective function value: ", round(objective_value(model2), digits=3))

    t+=1
end

MethodError: MethodError: Cannot `convert` an object of type Array{Float64, 3} to an object of type Float64

Closest candidates are:
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex}
   @ DualNumbers ~/.julia/packages/DualNumbers/5knFX/src/dual.jl:24
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:84
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


In [36]:
sent_nurses_contam

UndefVarError: UndefVarError: `sent_nurses_contam` not defined

In [32]:
sent_nurses_uncontam

UndefVarError: UndefVarError: `sent_nurses_uncontam` not defined

In [31]:
nurses_sent_tot

93-element Vector{Float64}:
    0.0
 2418.021240234375
 2531.5240166187286
 2612.778315077936
 3348.2851610520706
 2324.133915885918
 1319.79529846713
  972.2629621000748
 1003.2561386360983
  609.9147842830223
    ⋮
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0